In [1]:
import os

current_dir_path = os.getcwd()
current_file_path = os.path.abspath(current_dir_path)
current_dir_path = os.path.dirname(current_file_path)
parent_dir_path = os.path.dirname(current_dir_path)

os.chdir(current_dir_path)

import pickle
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from tdamapper.core import MapperAlgorithm
from tdamapper.cover import CubicalCover
from tdamapper.plot import MapperLayoutInteractive, MapperLayoutStatic
from tdamapper.clustering import FailSafeClustering
from sklearn import metrics

from functions import *
from chi import *
from regressionP import *
from models import *

In [2]:
data1 = pd.read_csv("./Data/NPA_TMA2_1.csv", low_memory=False)[:-2]
data2 = pd.read_csv("./Data/NPA_TMA2_2.csv", low_memory=False)[:-2]
data3 = pd.read_csv("./Data/NPA_TMA2_3.csv", low_memory=False)[:-2]
data4 = pd.read_csv("./Data/NPA_TMA2_4_new.csv", low_memory=False)[:-2]
data5 = pd.read_csv("./Data/NPA_TMA2_5.csv", low_memory=False)[:-2]
data6 = pd.read_csv("./Data/NPA_TMA2_6_new.csv", low_memory=False)[:-2]

dataA2 = pd.concat([data1, data2, data3, data4, data5, data6], ignore_index=True)

dataA1 = pd.read_csv("./Data/NPA_TMA1_new.csv")[:-2]

In [ ]:
# for i in select_lst:
#     print(dataA2[i].value_counts())

In [32]:
select_lst = [
    '道路類別-第1當事者-名稱', '速限-第1當事者', 
    '道路型態大類別名稱', '道路型態子類別名稱',
    '事故位置大類別名稱', '事故位置子類別名稱', 
    '事故類型及型態大類別名稱', '事故類型及型態子類別名稱',
    '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱',
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',
    '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分向設施子類別名稱',
    '車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    '死亡受傷人數'
]

dist_dfA1 = preprocess(dataA1, select_lst)
dist_dfA2 = preprocess(dataA2, select_lst)

rbind_data = pd.concat([dist_dfA1[0], dist_dfA2[0]], axis=0, ignore_index=True)
rbind_data['速限-第1當事者'] = rbind_data['速限-第1當事者'].apply(lambda x: 1 if x > 60 else 0)

select_lst.remove('死亡受傷人數')

rbind_data = pd.get_dummies(rbind_data[select_lst], columns=select_lst)

X1 = rbind_data.to_numpy()

In [33]:
pca = PCA(n_components=15)
lens1 = pca.fit_transform(X1)

# 查看每個主成分保留的變異量比例
explained_variance_ratio = pca.explained_variance_ratio_
print(sum(explained_variance_ratio))
print(explained_variance_ratio)

0.7968339279926917
[0.24139843 0.11647993 0.06990822 0.05897289 0.04888923 0.03367925
 0.03299789 0.03136332 0.02868323 0.02471669 0.02423999 0.02255262
 0.02212497 0.02188056 0.0189467 ]


In [ ]:
start_time = time.time()

lens1 = PCA(5).fit_transform(X1)

mapper_algo1 = MapperAlgorithm(
    cover = CubicalCover(
        n_intervals = 8,
        overlap_frac = 0.5
    ),
    clustering = FailSafeClustering(
        clustering = AgglomerativeClustering(3),
        verbose = False)
)

mapper_graph1 = mapper_algo1.fit_transform(X1, lens1)

end_time = time.time()
elapsed_time = end_time - start_time
print(elapsed_time)

In [ ]:
mapper_plot1 = MapperLayoutInteractive(
    mapper_graph1,
    colors = rbind_data[['道路型態大類別名稱_交岔路']].to_numpy(),
    cmap = 'jet',
    # agg = np.nanmean,
    agg = most_frequent_nonan,
    dim = 3,
    iterations = 30,
    seed = 6,
    width = 800,
    height = 500)

In [ ]:
fig_mean1 = mapper_plot1.plot()
fig_mean1.show(config={'scrollZoom': True})

In [8]:
# import pickle

# with open('CalculatedData/道路V2_dummy.pkl', 'wb') as f:
#     pickle.dump(mapper_graph1, f)

# 模型比較

In [49]:
# for i in rbind_data.columns:
#     print(i)

In [20]:
# List of columns to select
select_lst = [
    '道路類別-第1當事者-名稱', '速限-第1當事者', 
    '道路型態大類別名稱', '道路型態子類別名稱',
    '事故位置大類別名稱', '事故位置子類別名稱', 
    '事故類型及型態大類別名稱', '事故類型及型態子類別名稱',
    '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱',
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',
    '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分向設施子類別名稱',
    '車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    '死亡受傷人數'
]

dist_dfA1 = preprocess(dataA1, select_lst)
dist_dfA2 = preprocess(dataA2, select_lst)

rbind_data = pd.concat([dist_dfA1[0], dist_dfA2[0]], axis=0, ignore_index=True)
rbind_data['速限-第1當事者'] = rbind_data['速限-第1當事者'].apply(lambda x: 1 if x > 60 else 0)

select_lst.remove('死亡受傷人數')

dist_df = process_data(rbind_data)
scaler = StandardScaler()
full_dist = pd.DataFrame(scaler.fit_transform(dist_df), columns = dist_df.columns)
# rbind_data = pd.get_dummies(rbind_data[select_lst], columns=select_lst)

X1 = rbind_data.to_numpy()
# rbind_data

In [21]:
full_dist.columns

Index(['道路類別-第1當事者-名稱', '速限-第1當事者', '道路型態大類別名稱', '道路型態子類別名稱', '事故位置大類別名稱',
       '事故位置子類別名稱', '事故類型及型態大類別名稱', '事故類型及型態子類別名稱', '路面狀況-路面鋪裝名稱',
       '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱', '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱',
       '號誌-號誌種類名稱', '號誌-號誌動作名稱', '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分向設施子類別名稱',
       '車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
       '死亡', '受傷'],
      dtype='object')

In [30]:
# with open('CalculatedData/道路V2_dummy.pkl', 'rb') as f:
#     mapper_graph1 = pickle.load(f)
    
# mapper_plot1 = MapperLayoutInteractive(
#     mapper_graph1,
#     colors = full_dist[['號誌-號誌動作名稱']].to_numpy(),
#     cmap = 'jet',
#     # agg = most_frequent_nonan,
#     agg = np.mean,
#     dim = 3,
#     iterations = 30,
#     seed = 6,
#     width = 800,
#     height = 500)

# fig_mean1 = mapper_plot1.plot()
# fig_mean1.show(config={'scrollZoom': True})

In [21]:
x = vars(mapper_plot1._MapperLayoutInteractive__fig)['_data_objs'][1]['x']
y = vars(mapper_plot1._MapperLayoutInteractive__fig)['_data_objs'][1]['y']
z = vars(mapper_plot1._MapperLayoutInteractive__fig)['_data_objs'][1]['z']

threeDimData = pd.DataFrame({'x': x, 'y': y, 'z': z})

import re
data_tuple = vars(mapper_plot1._MapperLayoutInteractive__fig)['_data_objs'][1]['text']

data = []
for item in data_tuple:
    color = int(re.search(r'color: (-?\d+)', item).group(1))
    node = int(re.search(r'node: (\d+)', item).group(1))
    size = int(re.search(r'size: (\d+)', item).group(1))
    data.append({'color': color, 'node': node, 'size': size})
component_info = pd.DataFrame(data)

full_info = pd.concat([component_info, threeDimData], axis=1)

mp_content_origin = vars(mapper_plot1._MapperLayoutInteractive__graph)['_node']

mp_content = pd.DataFrame.from_dict(mp_content_origin, orient='index')
mp_content.reset_index(inplace=True)
mp_content.rename(columns={'index': 'node'}, inplace=True)

full_info = pd.merge(full_info, mp_content, on=['node', 'size'], how='inner')

In [27]:
calinski_data = get_calinski_from_db(full_info, 0.1)
labels = calinski_data[3]
db = calinski_data[2]
n_clusters_ = calinski_data[4]

print(n_clusters_)

# do_plot(full_info, calinski_data, labels, db, n_clusters_)

3


In [28]:
label_0 = full_info[full_info['label'] == 0]
label_1 = full_info[full_info['label'] == 1]
label_out = full_info[(full_info['label'] != 1) & (full_info['label'] != 0)]

count_0 = get_count_dict(label_0)
count_1 = get_count_dict(label_1)
count_out = get_count_dict(label_out)

full_0 = rbind_data.loc[count_0.keys()]
full_1 = rbind_data.loc[count_1.keys()]
# 離群值不需要被處理
full_out = rbind_data.loc[count_out.keys()]
lst01 = list(count_0.keys() & count_1.keys())
lsto0 = list(count_out.keys() & count_0.keys())
lsto1 = list(count_out.keys() & count_1.keys())
# Node
full_01 = full_0.loc[lst01]

full_combine = pd.concat([full_01], axis=0)
full_combine = full_combine.reset_index()
full_combine = full_combine.drop_duplicates(subset='index', keep='first')
full_combine = full_combine.drop('index', axis=1)
# 去掉連接點，使分析更嚴謹
full_0 = full_0.drop(lst01 + lsto0, errors='ignore')
full_1 = full_1.drop(lst01 + lsto1, errors='ignore')

print('01連接點數量', len(lst01))
print('o0連接點數量', len(lsto0))
print('o1連接點數量', len(lsto1))
print('離群值數量', full_out.shape[0])

full_combine.shape[0] + full_0.shape[0] + full_1.shape[0] + full_out.shape[0] == rbind_data.shape[0]

01連接點數量 100
o0連接點數量 4063
o1連接點數量 2740
離群值數量 55415


False

In [58]:
# lst_logit = [
#     '路面狀況-路面狀態名稱', '當事者屬-性-別名稱', '當事者事故發生時年齡', 
#     '車輛撞擊部位大類別名稱-最初', 
#     '光線名稱', '道路類別-第1當事者-名稱',
#     '速限-第1當事者', '道路型態大類別名稱', '事故位置大類別名稱', 
#     '號誌-號誌種類名稱', '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分道設施-快車道或一般車道間名稱',
#     '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
#     '事故類型及型態大類別名稱', '事故位置子類別名稱', '事故類型及型態子類別名稱', 
#     '當事者行動狀態子類別名稱', '車輛撞擊部位子類別名稱-最初', '肇因研判子類別名稱-個別',
#     '當事者區分-類別-子類別名稱-車種', '保護裝備名稱', '行動電話或電腦或其他相類功能裝置名稱', 
#     '當事者行動狀態大類別名稱', '車輛撞擊部位大類別名稱-其他', 
#     '肇因研判大類別名稱-個別', '肇事逃逸類別名稱-是否肇逃', '路面狀況-路面鋪裝名稱', 
#     # '車道劃分設施-分向設施子類別名稱', '道路障礙-障礙物名稱', '車輛撞擊部位子類別名稱-其他',

#     # '號誌-號誌動作名稱', '當事者區分-類別-大類別名稱-車種', '肇因研判大類別名稱-主要', # 降低預測值
#     # '道路障礙-視距名稱', '車道劃分設施-分道設施-快慢車道間名稱', '車輛撞擊部位大類別名稱-其他', # 降低預測值
#     # '道路型態子類別名稱', '路面狀況-路面缺陷名稱', '天候名稱' # 降低
# ]

# full_combine_X, full_combine_y = get_clusterN_logit(full_combine, lst_logit)
# full_0_X, full_0_y = get_clusterN_logit(full_0, lst_logit)
# full_1_X, full_1_y = get_clusterN_logit(full_1, lst_logit)
# full_out_X, full_out_y = get_clusterN_logit(full_out, lst_logit)

# print(full_combine_y.value_counts())
# print(full_out_y.value_counts())
# print(full_0_y.value_counts())
# print(full_1_y.value_counts())

# 顯著特徵

In [43]:
def table(colnames, full_0, full_1):
    
    combined_df = pd.concat([full_0[colnames].value_counts(normalize = True), 
                             full_1[colnames].value_counts(normalize = True)
                            ],
                            axis=1).fillna(0)

    combined_df.columns = ['A', 'B']
    
    return combined_df

In [67]:
select_lst = [
    '道路類別-第1當事者-名稱', '速限-第1當事者', 
    '道路型態大類別名稱', '道路型態子類別名稱',
    '事故位置大類別名稱', '事故位置子類別名稱', 
    '事故類型及型態大類別名稱', '事故類型及型態子類別名稱',
    '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱',
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',
    '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分向設施子類別名稱',
    '車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    # '死亡受傷人數'
]

full_0['label'] = 0
full_1['label'] = 1

labeled_data = pd.concat([full_0, full_1])
labeled_data.reset_index(drop=True, inplace=True)


def get_clusterN_logit(cluster_data, lst):
    scaler = StandardScaler()
    
    c0_for_lm = process_data(cluster_data)
    c0_for_lm_X = pd.DataFrame(scaler.fit_transform(c0_for_lm), columns=c0_for_lm.columns).reset_index(drop=True, inplace=False)
    # label設定
    c0_for_lm_y = cluster_data.apply(lambda row: 1 if row['label'] != 0 else 2, axis=1)
    # c0_for_lm_y = cluster_data['label']
        
    c0_for_lm_X = c0_for_lm_X[lst]
    
    return c0_for_lm_X, c0_for_lm_y


X, y = get_clusterN_logit(labeled_data, select_lst)

In [68]:
import statsmodels.api as sm

X_train, X_test, y_train, y_test = train_test_split(X, y.replace(2, 0), test_size=0.2, random_state=42)

logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()

print(result.summary())

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
accuracy = log_reg.score(X_test, y_test)

print(f"Accuracy: {accuracy:.2f}")

Optimization terminated successfully.
         Current function value: 0.000482
         Iterations 15
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:               133260
Model:                          Logit   Df Residuals:                   133239
Method:                           MLE   Df Model:                           20
Date:                Mon, 26 Aug 2024   Pseudo R-squ.:                  0.9993
Time:                        00:31:27   Log-Likelihood:                -64.237
converged:                       True   LL-Null:                       -89278.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
道路類別-第1當事者-名稱              -0.2692      0.370     -0.727      0.467      -0.995  